## Testing the FashionClip package for image characterization
https://github.com/patrickjohncyh/fashion-clip

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fashion_clip.fashion_clip import FashionCLIP

In [ ]:
# Get images
from pathlib import Path
image_paths = Path('../data/farfetch/images').glob('*.jpg')
image_list = [str(p) for p in image_paths]
print(f"Found {len(image_list)} images, ", image_list[0])

In [ ]:
# Or when initializing FashionCLIP
fclip = FashionCLIP('fashion-clip')
# Ensure model is in float32
fclip.model.half()

In [ ]:
# we create image embeddings and text embeddings
image_embeddings = fclip.encode_images(image_list, batch_size=32)
# text_embeddings = fclip.encode_text(texts, batch_size=32)

In [ ]:
image_embeddings

### Test weaviate search

Taking one image of the H&M catalogue and looking for similar items in Farfetch!

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)

In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
)

client.is_ready()

In [ ]:
images = [{
    "image_name": img,
    "fashion_clip_vector": emb.tolist(),
} for img, emb in zip(image_list, image_embeddings)]

In [ ]:
import weaviate.classes.config as wc

if client.collections.exists("Farfetch2"):
    client.collections.delete("Farfetch2")

client.collections.create(
    name="Farfetch2",
    properties=[
        wc.Property(name="image_name", data_type=wc.DataType.TEXT),
    ],
    vector_config=[
        wc.Configure.Vectors.self_provided(
            name="fashion_clip_vector",
            vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE
            )
        )
    ]
)

In [ ]:
farfetch_collection = client.collections.get("Farfetch2")

with farfetch_collection.batch.dynamic() as batch:
    for img_data in images:
        batch.add_object(
            properties={"image_name": img_data["image_name"]}, 
            vector=img_data["fashion_clip_vector"]
        )

In [ ]:
image1 =  "../data/h-and-m-personalized-fashion-recommendations/images/093/0930409001.jpg"
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig, axes = plt.subplots(1, 1, figsize=(3, 3))

img = mpimg.imread(image1)
axes.imshow(img)
axes.axis('off')  

plt.tight_layout()
plt.show()

In [ ]:
img.shape

In [ ]:
query_embedding = fclip.encode_images([image1], batch_size=1)
query_embedding

In [ ]:
response = farfetch_collection.query.near_vector(
    near_vector=query_embedding[0],
    limit=5
)

# Print results
for obj in response.objects:
    print(obj.properties)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
axes = axes.ravel()

for i, ax in enumerate(axes):
    img = response.objects[i].properties["image_name"]
    img = mpimg.imread(img)
    ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()